In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas
import numpy as np
!pip install nlpaug
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install transformers 

In [3]:
import pickle
with open('drive/MyDrive/CS247/train_data.pkl', 'rb') as fid:
     data= pickle.load(fid)
nan_value = float("NaN")
data.replace("", nan_value, inplace=True)
data.dropna(inplace=True)


In [4]:
toxic = data.loc[data["toxic"] == 1]
severe_toxic = data.loc[data["severe_toxic"]==1]
insult = data.loc[data["insult"] == 1]
obscene = data.loc[data["obscene"] == 1]
threat = data.loc[data["threat"] == 1]
identity_hate = data.loc[data["identity_hate"] ==1]

In [5]:
df_negative_all= data.loc[(data['obscene'] == 0) & (data['toxic'] ==0) & (data['severe_toxic'] ==0)& (data['insult'] == 0) & (data['threat'] == 0) & (data['identity_hate'] == 0) ]


In [6]:
import plotly.express as px
labels = ['Toxic', 'Severe toxic', 'Insult', 'Obscene', 'Threat', 'Identity hate','Negative_samples']
colors = ['blue'] * 7
colors[6] = 'crimson'
fig = px.bar(x=labels,y=[len(toxic),len(severe_toxic),len(insult),len(obscene),len(threat),len(identity_hate),len(df_negative_all)],color=colors)
fig.show()

In [7]:
labels = ['toxic', 'severe_toxic', 'insult', 'obscene', 'threat', 'identity_hate']
for label in labels:
    df_neg = data.loc[data[label] == 0]
    colors = ['blue','red']
    fig = px.bar(x=[label,'not ' + label.lower()],y=[(len(data) - len(df_neg)),len(df_neg)],color=colors)
    fig.show()

In [9]:
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="insert",device='cuda',stopwords=stop_words,aug_p=0.4)

augsubs = naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', action="substitute",device='cuda',stopwords=stop_words,aug_p=0.4)
aug_syn = naw.SynonymAug(aug_src='wordnet')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

In [10]:

#  max length for text 
def insert(x):
    return aug.augment(x)

def substitute(x):
    return augsubs.augment(x)



In [11]:
labels = ['toxic', 'severe_toxic',  'obscene', 'threat', 'insult','identity_hate']
xtrain = data['comment'].tolist()
y_toxic_train =data['toxic'].tolist()
y_severe_toxic_train = data['severe_toxic'].tolist()
y_obscene_train = data['obscene'].tolist()
y_threat_train = data['threat'].tolist()
y_insult_train = data['insult'].tolist()
y_identity_hate_train = data['identity_hate'].tolist()
xs = [(toxic,y_toxic_train),(severe_toxic,y_severe_toxic_train),(obscene,y_obscene_train),(threat,y_threat_train),(insult,y_insult_train),(identity_hate,y_identity_hate_train)]
k = 0
for text,target in xs:
    xtemp = xtrain.copy()
    for i in range(len(text)):
        comment = text['comment'].iloc[i]
        if len(comment)>2000:
          continue
        augmented_comment = insert(comment)
        augmentedsubs_comment = substitute(comment)
        xtemp.append(augmented_comment)
        xtemp.append(augmentedsubs_comment)
        target.append(1)
        target.append(1)

    with open(labels[k] + '_train.pkl' ,'wb') as f:
         pickle.dump(xtemp, f)
    with open(labels[k] + '_labels.pkl','wb') as f:
         pickle.dump(target, f)
    k+=1
    
    
